Отлично, делаю **один большой файл** в формате *Markdown с HTML-вставками* — так его можно вставить в `README.md`, ноутбук или любую Markdown-страницу.
Код — **рабочий**, описания — **подробные (уровень PyTorch/TF)**. Хочешь — просто удаляй HTML-блоки и остаётся чистый Python.

---

# `universal_competition_toolkit_MAXPRO.md`  (вставь содержимое целиком)

```markdown
<!-- ============================== -->
<!-- UNIVERSAL COMPETITION TOOLKIT -->
<!-- MAX PRO — ОДИН ФАЙЛ           -->
<!-- ============================== -->

<div style="padding:16px;border:1px solid #444;border-radius:10px;background:#0f0f0f;color:#eaeaea">
<h1>🎯 UNIVERSAL COMPETITION TOOLKIT — MAX PRO (Один файл)</h1>
<p>
Готовый, универсальный, <b>боевой</b> набор для соревнований (VSERO/FAIO/Kaggle/Инфометрикс).<br>
Секции идут в <b>практическом порядке</b>: от импорта и загрузки данных до предобработки, обучения и сабмитов.<br>
<b>Все описания в HTML-блоках</b> — их можно удалить, оставив только рабочий код.
</p>

<ul>
<li><a href="#sec1">1. Импорты, утилиты, seed, логгер</a></li>
<li><a href="#sec2">2. Загрузка данных (CSV/JSON/Excel/Parquet/Pickle/Images/Audio)</a></li>
<li><a href="#sec3">3. Предобработка текста (чистка, токенизация, лемматизация, n-граммы)</a></li>
<li><a href="#sec4">4. Предобработка изображений (PIL/OpenCV/torchvision + аугментации)</a></li>
<li><a href="#sec5">5. Предобработка табличных данных (категориальные/числовые/скейлинг)</a></li>
<li><a href="#sec6">6. Базовые модели (LightGBM/XGBoost/CatBoost + Torch NN скелет)</a></li>
<li><a href="#sec7">7. Эмбеддинги (SBERT/LaBSE/CLIP) и ретривл</a></li>
<li><a href="#sec8">8. Сабмиты (классификация/регрессия/текст)</a></li>
<li><a href="#sec9">9. Утилиты (профилирование, сохранение, контроль версий)</a></li>
</ul>
</div>
```

---

## 1) Импорты, утилиты, seed, логгер <a id="sec1"></a>

```markdown
<div style="border:1px solid #333;padding:14px;border-radius:8px;background:#111;color:#ddd">
<h2>📦 1. Импорты (минимально необходимые)</h2>
<p><b>Назначение:</b> базовый набор для любых задач: файловая система, массивы/таблицы, типы, логи.</p>
<p><b>Когда использовать:</b> всегда. Это фундамент пайплайна.</p>
<p><b>Где ломается на кластерах (VSERO):</b> иногда отсутствуют доп.библиотеки (lightgbm/catboost/spacy). Ставь условные импорты.</p>
</div>
```

```python
import os, sys, json, math, random, shutil, warnings, pathlib, traceback, gc, time
from glob import glob
from typing import List, Dict, Any, Tuple, Optional, Iterable

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")
print("✔️ Импорты загружены")
```

```markdown
<div style="border:1px solid #333;padding:14px;border-radius:8px;background:#0f0f0f;color:#ddd">
<h3>📁 Утилиты путей</h3>
<p>Нормализуют пути, создают директории, собирают списки файлов (картинок/аудио). Особенно полезно на кластерах, где рабочая директория меняется.</p>
</div>
```

```python
def ensure_dir(path: str) -> str:
    p = pathlib.Path(path); p.mkdir(parents=True, exist_ok=True); return str(p)

def norm_path(path: str) -> str:
    return str(pathlib.Path(path).expanduser().resolve())

def list_images(directory: str) -> List[str]:
    exts = ["*.png","*.jpg","*.jpeg","*.bmp","*.gif","*.webp"]
    files=[]
    for e in exts: files.extend(glob(os.path.join(directory, e)))
    return sorted(files)

def list_audio(directory: str) -> List[str]:
    exts = ["*.wav","*.mp3","*.ogg","*.flac","*.m4a"]
    files=[]
    for e in exts: files.extend(glob(os.path.join(directory, e)))
    return sorted(files)

def file_size(path: str) -> str:
    size = os.path.getsize(path); units = ["B","KB","MB","GB","TB"]
    for u in units:
        if size < 1024: return f"{size:.2f}{u}"
        size /= 1024
```

```markdown
<div style="border:1px solid #333;padding:14px;border-radius:8px;background:#0d0d0d;color:#ddd">
<h3>🎲 Seed (фиксируем случайность)</h3>
<p><b>Зачем:</b> воспроизводимость результатов на FAIO/VSERO/Kaggle.<br>
<b>Подводные камни:</b> в torch отключаем CUDNN benchmark для детерминизма (может замедлить).</p>
</div>
```

```python
def seed_everything(seed: int = 42):
    random.seed(seed); np.random.seed(seed)
    try:
        import torch
        torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    except Exception:
        pass
    print(f"🔒 Seed установлен: {seed}")

seed_everything(42)
```

```markdown
<div style="border:1px solid #444;padding:14px;border-radius:8px;background:#101010;color:#ddd">
<h3>🧩 Логгер (цветной)</h3>
<p>Аккуратные логи с таймштампом. Чем сложнее пайплайн — тем важнее логгер.</p>
<p><b>Примечание:</b> модуль <code>termcolor</code> может отсутствовать. Если нет — делаем graceful fallback.</p>
</div>
```

```python
import datetime
try:
    from termcolor import colored
    def _c(text, color): return colored(text, color)
except Exception:
    def _c(text, color): return text

class Log:
    def __init__(self, logfile: Optional[str] = None):
        self.logfile = logfile

    def _write(self, line: str):
        if self.logfile:
            with open(self.logfile, "a", encoding="utf-8") as f: f.write(line+"\n")

    def _ts(self): return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    def info(self, msg: str):    line=f"[{self._ts()}] [INFO] {msg}";    print(_c(line,"cyan"));    self._write(line)
    def success(self, msg: str): line=f"[{self._ts()}] [OK] {msg}";      print(_c(line,"green"));   self._write(line)
    def warn(self, msg: str):    line=f"[{self._ts()}] [WARN] {msg}";    print(_c(line,"yellow"));  self._write(line)
    def error(self, msg: str):   line=f"[{self._ts()}] [ERR] {msg}";     print(_c(line,"red"));     self._write(line)

logger = Log()
logger.success("Логгер инициализирован")
```

---

## 2) Загрузка данных (CSV/JSON/Excel/Parquet/Pickle/Images/Audio) <a id="sec2"></a>

```markdown
<div style="border:1px solid #333;padding:14px;border-radius:8px;background:#0f0f0f;color:#ddd">
<h2>📥 2. Загрузка данных</h2>
<p><b>Цель:</b> единый интерфейс для всех форматов. Код пишет логи, ловит частые ошибки и даёт советы (особенно для VSERO/FAIO кластеров).</p>
<p><b>Ключевые темы:</b> когда <code>read_csv</code> использовать нельзя; ускорение чтения больших файлов; Parquet на соревнованиях; избегаем UTF-8 ошибок; пакетная загрузка изображений.</p>
</div>
```

### 2.1 CSV / JSON / Excel / Parquet / Pickle

```python
def load_csv(path: str, low_memory: bool = True, encoding: Optional[str] = "utf-8",
             sep: str = ",", nrows: Optional[int] = None) -> pd.DataFrame:
    """
    Быстрая и безопасная загрузка CSV.
    Советы:
    - Если 'utf-8' падает: попробуй encoding='utf-8-sig' или 'cp1251'
    - Для огромных файлов: укажи dtype столбцов, usecols, nrows
    - На кластерах: low_memory=True снижает RAM, но типы могут стать 'object'
    """
    logger.info(f"Читаю CSV: {path}")
    try:
        df = pd.read_csv(path, sep=sep, encoding=encoding, low_memory=low_memory, nrows=nrows)
        logger.success(f"CSV OK: shape={df.shape}")
        return df
    except UnicodeDecodeError as e:
        logger.warn("UnicodeDecodeError → пробую 'utf-8-sig'")
        try:
            df = pd.read_csv(path, sep=sep, encoding="utf-8-sig", low_memory=low_memory, nrows=nrows)
            logger.success(f"CSV OK (utf-8-sig): shape={df.shape}")
            return df
        except Exception as e2:
            logger.error(f"Проблема с кодировкой: {e2}")
            raise
    except pd.errors.ParserError as e:
        logger.warn("ParserError: укажи sep=';' или engine='python'")
        df = pd.read_csv(path, sep=sep, encoding=encoding, low_memory=low_memory,
                         engine="python", nrows=nrows)
        logger.success(f"CSV OK (engine=python): shape={df.shape}")
        return df

def load_json(path: str, lines: bool = False) -> Any:
    """
    JSON:
    - lines=True для JSONL (по строке на объект)
    - иначе стандартный JSON
    """
    logger.info(f"Читаю JSON: {path} (lines={lines})")
    if lines:
        return pd.read_json(path, lines=True)
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def load_excel(path: str, sheet_name: Optional[str] = None) -> pd.DataFrame:
    logger.info(f"Читаю Excel: {path} sheet={sheet_name or '(первый)'}")
    return pd.read_excel(path, sheet_name=sheet_name)

def load_parquet(path: str, columns: Optional[List[str]] = None) -> pd.DataFrame:
    """
    Parquet:
    - быстрее и компактнее CSV
    - на соревнованиях, если дают parquet — используй его, экономит время
    """
    logger.info(f"Читаю Parquet: {path} cols={columns}")
    return pd.read_parquet(path, columns=columns)

def load_pickle(path: str) -> Any:
    logger.info(f"Читаю Pickle: {path}")
    import pickle
    with open(path, "rb") as f:
        return pickle.load(f)
```

```markdown
<div style="border:1px solid #444;padding:14px;border-radius:8px;background:#121212;color:#ddd">
<h4>⚠️ Когда <code>read_csv</code> использовать нельзя</h4>
<ul>
<li>Файл очень большой (5–20 ГБ) → разорвёт RAM. Лучше <b>читать по частям</b> (chunksize) или перейти на Parquet.</li>
<li>Сложные типы (списки/словарики внутри ячеек) → удобнее JSONL (lines=True) или Pickle.</li>
<li>Нестабильная кодировка (много смешанных кодировок) → сначала нормализуй (iconv, recode), либо читай двоично, чисти, потом парсь.</li>
</ul>

<h4>💡 Ускорение чтения больших CSV</h4>
<ul>
<li><code>usecols</code>: выбирай только нужные колонки</li>
<li><code>dtype</code>: сразу укажи типы столбцов</li>
<li><code>chunksize</code>: читай кусками и конкатенируй</li>
<li>Если доступен Parquet → конвертируй один раз и читай Parquet дальше</li>
</ul>

<pre><code># пример чтения по частям
chunks = []
for chunk in pd.read_csv("huge.csv", chunksize=200_000, usecols=["id","text","label"]):
    # предобработай chunk при необходимости
    chunks.append(chunk)
df = pd.concat(chunks, ignore_index=True)
</code></pre>

<h4>🧯 Как избежать UTF-8 ошибок</h4>
<p>Если падает на <code>utf-8</code>: попробуй <code>utf-8-sig</code>, далее <code>cp1251</code>. Если всё плохо — открой в бинарном режиме, посмотри первые байты, определи BOM.</p>

<h4>🏁 Parquet на соревнованиях</h4>
<p>Где доступно — используем Parquet: быстрее чтение/запись, меньше I/O. На VSERO кластер часто это существенно сокращает время.</p>
</div>
```

### 2.2 Загрузка изображений (PIL/OpenCV/torchvision) + пакетная загрузка

```python
from PIL import Image

def load_image_pil(path: str, mode: str = "RGB") -> Image.Image:
    """
    Безопасная загрузка картинки через PIL.
    mode='RGB' гарантирует 3 канала.
    """
    img = Image.open(path)
    if mode: img = img.convert(mode)
    return img

def load_image_cv2(path: str, to_rgb: bool = True):
    """
    cv2.imread → BGR; если to_rgb=True, преобразуем к RGB.
    cv2 быстрее на многих кластерах, но зависит от сборки.
    """
    import cv2
    im = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if im is None:
        raise FileNotFoundError(f"Не удалось прочитать: {path}")
    if to_rgb and len(im.shape) == 3:
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    return im

def iter_image_batches(paths: List[str], batch_size: int = 32, loader="pil"):
    """
    Пакетная загрузка изображений — чтобы не съедать всю RAM.
    loader: 'pil' | 'cv2'
    """
    assert loader in ("pil","cv2")
    for i in range(0, len(paths), batch_size):
        batch_paths = paths[i:i+batch_size]
        batch=[]
        for p in batch_paths:
            try:
                if loader=="pil":
                    batch.append(load_image_pil(p, mode="RGB"))
                else:
                    batch.append(load_image_cv2(p, to_rgb=True))
            except Exception as e:
                logger.warn(f"Пропускаю {p}: {e}")
        yield batch_paths, batch
```

```markdown
<div style="border:1px solid #444;padding:14px;border-radius:8px;background:#121212;color:#ddd">
<h4>🧠 Как правильно грузить изображения партиями</h4>
<ul>
<li>Не загружай сразу все 100k изображений — выбьет память.</li>
<li>Используй <code>iter_image_batches</code> и обрабатывай по 32–128 картинок.</li>
<li>Закрывай дескрипторы (для PIL достаточно убрать ссылки, периодически делай <code>gc.collect()</code>).</li>
</ul>

<h4>⏩ Ускорение</h4>
<ul>
<li>cv2 обычно быстрее, но убедись в корректности каналов (BGR→RGB)</li>
<li>Храни промежуточные преобразования (кеш) при длительных экспериментах</li>
</ul>
</div>
```

### 2.3 Аудио (librosa)

```python
def load_audio(path: str, sr: Optional[int] = None):
    """
    Читает аудио через librosa. Если sr=None — родная частота.
    Потребуется: pip install librosa (может отсутствовать на кластере)
    """
    try:
        import librosa
    except Exception as e:
        logger.error("librosa не установлен. Установи или отключи аудио.")
        raise
    y, current_sr = librosa.load(path, sr=sr, mono=True)
    return y, (sr or current_sr)
```

---

## 3) Предобработка текста <a id="sec3"></a>

```markdown
<div style="border:1px solid #333;padding:14px;border-radius:8px;background:#0f0f0f;color:#ddd">
<h2>📝 3. Предобработка текста</h2>
<p>Чистка, унификация, токенизация, лемматизация. Делай минимум на старте: lower/trim/нормализация unicode.</p>
<p><b>На кластерах:</b> NLTK/spaCy модели могут отсутствовать. Используй try/except и degrade gracefully.</p>
</div>
```

```python
import re
import unicodedata

def text_basic_clean(s: Any) -> str:
    if not isinstance(s, str): return ""
    s = s.strip()
    s = unicodedata.normalize("NFKC", s)   # unicode нормализация
    s = s.replace("\u200b","")             # zero-width space
    s = re.sub(r"\s+", " ", s)             # мульти-пробелы -> один
    return s.lower()

def clean_text_column(df: pd.DataFrame, col: str, new_col: Optional[str] = None) -> pd.DataFrame:
    new_col = new_col or (col + "_clean")
    df[new_col] = df[col].apply(text_basic_clean)
    return df
```

```markdown
<div style="border:1px solid #444;padding:14px;border-radius:8px;background:#121212;color:#ddd">
<h4>🔤 Токенизация / лемматизация (NLTK/spaCy)</h4>
<p>Используй, если доступно. Если нет — падай на базовую чистку.</p>
</div>
```

```python
def try_spacy_lemmatize(texts: List[str], lang: str = "ru"):
    """
    Пытаемся лемматизировать через spaCy. Если модели нет — возвращаем исходный список.
    Требуется: python -m spacy download ru_core_news_sm (или en_core_web_sm)
    """
    try:
        import spacy
        if lang == "ru":
            nlp = spacy.load("ru_core_news_sm")
        elif lang == "en":
            nlp = spacy.load("en_core_web_sm")
        else:
            return texts
        out=[]
        for doc in nlp.pipe(texts, batch_size=256, disable=["ner","parser"]):
            out.append(" ".join([t.lemma_ for t in doc]))
        return out
    except Exception as e:
        logger.warn(f"spaCy недоступен: {e}")
        return texts

def try_nltk_tokenize(text: str) -> List[str]:
    try:
        import nltk
        from nltk.tokenize import word_tokenize
        # nltk.download('punkt') # однажды на машине с сетью
        return word_tokenize(text)
    except Exception:
        return text.split()
```

```markdown
<div style="border:1px solid #444;padding:14px;border-radius:8px;background:#121212;color:#ddd">
<h4>📌 N-граммы (универсальный шаблон)</h4>
<p>Применим к классификации, TF-IDF, базовым моделям.</p>
</div>
```

```python
from collections import Counter

def generate_ngrams(tokens: List[str], n: int = 2) -> List[Tuple[str,...]]:
    return [tuple(tokens[i:i+n]) for i in range(0, max(0, len(tokens)-n+1))]

def top_ngrams(texts: Iterable[str], n: int = 2, top_k: int = 20) -> List[Tuple[Tuple[str,...], int]]:
    cnt = Counter()
    for s in texts:
        toks = try_nltk_tokenize(text_basic_clean(s))
        for g in generate_ngrams(toks, n=n): cnt[g]+=1
    return cnt.most_common(top_k)
```

---

## 4) Предобработка изображений (PIL/OpenCV/torchvision + аугментации) <a id="sec4"></a>

```markdown
<div style="border:1px solid #333;padding:14px;border-radius:8px;background:#0f0f0f;color:#ddd">
<h2>🖼️ 4. Предобработка изображений</h2>
<p>Шаблоны для baseline: resize, центр/рандом кропы, нормализация, аугментации.</p>
<p><b>Замечание:</b> torchvision может отсутствовать на кластере. Делаем try/except и альтернативы (PIL/cv2).</p>
</div>
```

```python
# torchvision-пайплайн (если доступен)
try:
    import torch
    from torchvision import transforms

    def get_torchvision_train_transforms(img_size: int = 224):
        return transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomResizedCrop(img_size, scale=(0.8,1.0)),
            transforms.ToTensor(),
            # нормализация под ImageNet
            transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
        ])

    def get_torchvision_valid_transforms(img_size: int = 224):
        return transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.CenterCrop(img_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
        ])
except Exception as e:
    logger.warn(f"torchvision недоступен: {e}")
    torch = None
```

```python
# Альтернативы на PIL/cv2 (минимум)
def pil_resize(img: Image.Image, size: Tuple[int,int]=(224,224)) -> Image.Image:
    return img.resize(size, Image.BILINEAR)

def cv2_resize(im, size: Tuple[int,int]=(224,224)):
    import cv2
    return cv2.resize(im, size, interpolation=cv2.INTER_LINEAR)
```

```markdown
<div style="border:1px solid #444;padding:14px;border-radius:8px;background:#121212;color:#ddd">
<h4>💡 Аугментации</h4>
<ul>
<li><b>Лёгкие</b>: flip, crop, color jitter (если классический классификатор)</li>
<li><b>Осторожно</b> с сильными аугментациями (mixup/cutout) на маленьких датасетах — легко переусердствовать</li>
</ul>
</div>
```

---

## 5) Предобработка табличных данных <a id="sec5"></a>

```markdown
<div style="border:1px solid #333;padding:14px;border-radius:8px;background:#0f0f0f;color:#ddd">
<h2>📊 5. Табличные признаки</h2>
<p>Скейлинг, one-hot, обработка пропусков, label encoding. Совместимо со sklearn/LightGBM/CatBoost.</p>
</div>
```

```python
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

def split_train_val(df: pd.DataFrame, target: str, test_size: float=0.2, random_state: int=42):
    X = df.drop(columns=[target]); y = df[target]
    Xtr, Xva, ytr, yva = train_test_split(X, y, test_size=test_size, random_state=random_state, shuffle=True, stratify=None)
    return Xtr, Xva, ytr, yva

def build_preprocessor(numeric_cols: List[str], cat_cols: List[str], scale: str="standard"):
    scaler = StandardScaler() if scale=="standard" else MinMaxScaler()
    cat = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    return ColumnTransformer(
        transformers=[
            ("num", scaler, numeric_cols),
            ("cat", cat, cat_cols)
        ],
        remainder="drop"
    )
```

---

## 6) Базовые модели (LightGBM/XGBoost/CatBoost + Torch NN скелет) <a id="sec6"></a>

```markdown
<div style="border:1px solid #333;padding:14px;border-radius:8px;background:#0f0f0f;color:#ddd">
<h2>🤖 6. Базовые модели</h2>
<p>Быстрые бэйслайны для табличных данных + скелет Torch-сети для кастомных задач.</p>
<p><b>На кластерах:</b> библиотеки бустингов могут отсутствовать — делай try/except и graceful degrade.</p>
</div>
```

```python
# LightGBM
def train_lgbm(Xtr, ytr, Xva, yva, params: Optional[Dict[str,Any]] = None):
    try:
        import lightgbm as lgb
    except Exception as e:
        logger.error("LightGBM не установлен"); raise
    params = params or dict(
        objective="regression",
        learning_rate=0.05,
        num_leaves=31,
        feature_fraction=0.9,
        bagging_fraction=0.8,
        bagging_freq=1,
        metric="rmse",
        n_estimators=2000
    )
    model = lgb.LGBMModel(**params)
    model.fit(Xtr, ytr, eval_set=[(Xva, yva)], eval_metric="rmse",
              callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=False)])
    return model

# XGBoost
def train_xgb(Xtr, ytr, Xva, yva, params: Optional[Dict[str,Any]] = None):
    try:
        import xgboost as xgb
    except Exception as e:
        logger.error("XGBoost не установлен"); raise
    params = params or dict(
        objective="reg:squarederror",
        eta=0.05,
        max_depth=8,
        subsample=0.8,
        colsample_bytree=0.8,
        n_estimators=2000,
        tree_method="hist"
    )
    model = xgb.XGBRegressor(**params)
    model.fit(Xtr, ytr, eval_set=[(Xva, yva)], eval_metric="rmse", verbose=False)
    return model

# CatBoost
def train_cat(Xtr, ytr, Xva, yva, params: Optional[Dict[str,Any]] = None):
    try:
        from catboost import CatBoostRegressor
    except Exception as e:
        logger.error("CatBoost не установлен"); raise
    params = params or dict(
        loss_function="RMSE",
        learning_rate=0.05,
        depth=8,
        iterations=5000,
        od_type="Iter",
        od_wait=200,
        verbose=False
    )
    model = CatBoostRegressor(**params)
    model.fit(Xtr, ytr, eval_set=(Xva, yva), use_best_model=True)
    return model
```

```python
# Torch NN (скелет)
def build_torch_mlp(input_dim: int, hidden: int = 256, out_dim: int = 1):
    try:
        import torch
        import torch.nn as nn
    except Exception as e:
        logger.error("PyTorch не установлен"); raise

    class MLP(nn.Module):
        def __init__(self):
            super().__init__()
            self.net = nn.Sequential(
                nn.Linear(input_dim, hidden),
                nn.ReLU(),
                nn.Dropout(0.2),
                nn.Linear(hidden, hidden),
                nn.ReLU(),
                nn.Dropout(0.2),
                nn.Linear(hidden, out_dim)
            )
        def forward(self, x): return self.net(x)
    return MLP()
```

---

## 7) Эмбеддинги и ретривл (SBERT/LaBSE/CLIP) <a id="sec7"></a>

```markdown
<div style="border:1px solid #333;padding:14px;border-radius:8px;background:#0f0f0f;color:#ddd">
<h2>🧠 7. Эмбеддинги</h2>
<p>Универсальные заготовки для извлечения эмбеддингов текста/картинок и простого поиска ближайших соседей.</p>
<p><b>На кластерах:</b> модельки из <code>sentence-transformers</code> могут быть недоступны оффлайн. Готовь локальные веса заранее.</p>
</div>
```

```python
def sbert_encode(texts: List[str], model_name: str = "sentence-transformers/all-MiniLM-L6-v2",
                 batch_size: int = 64) -> np.ndarray:
    try:
        from sentence_transformers import SentenceTransformer
    except Exception as e:
        logger.error("sentence-transformers не установлен"); raise
    model = SentenceTransformer(model_name)
    embs = model.encode(texts, batch_size=batch_size, convert_to_numpy=True, show_progress_bar=False, normalize_embeddings=True)
    return embs

def cosine_search(query_vec: np.ndarray, index_vecs: np.ndarray, top_k: int = 5) -> List[Tuple[int,float]]:
    # нормализованные косинусы → arg top-k
    sims = index_vecs @ query_vec
    idx = np.argpartition(-sims, top_k)[:top_k]
    idx = idx[np.argsort(-sims[idx])]
    return [(int(i), float(sims[i])) for i in idx]
```

```markdown
<div style="border:1px solid #444;padding:14px;border-radius:8px;background:#121212;color:#ddd">
<h4>🖼️ CLIP (zero-shot)</h4>
<p>Если есть доступ к OpenAI CLIP (или open-clip) оффлайн — можно делать zero-shot классификацию через текстовые промпты.</p>
</div>
```

```python
def clip_zero_shot_scores(images: List[Image.Image], texts: List[str], model_name: str = "ViT-B-32"):
    """
    Заготовка: используй open-clip / clip по доступности.
    Для краткости тут только схема — подключай реальную либу на своей машине.
    """
    try:
        import torch
        import clip  # openai/clip
    except Exception as e:
        logger.warn("CLIP недоступен. Подключи библиотеку и веса локально.")
        return None
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, preprocess = clip.load(model_name, device=device)
    with torch.no_grad():
        image_t = torch.stack([preprocess(im) for im in images]).to(device)
        text_t = clip.tokenize(texts).to(device)
        image_features = model.encode_image(image_t)
        text_features  = model.encode_text(text_t)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features  /= text_features.norm(dim=-1, keepdim=True)
        logits = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    return logits.cpu().numpy()  # [N_images, N_texts]
```

---

## 8) Сабмиты (классификация/регрессия/текст) <a id="sec8"></a>

```markdown
<div style="border:1px solid #333;padding:14px;border-radius:8px;background:#0f0f0f;color:#ddd">
<h2>📤 8. Сабмиты</h2>
<p>Три шаблона под самые частые форматы соревнований. Всегда проверяй <code>sample_submission.csv</code>.</p>
</div>
```

```python
def save_classification_submission(test_df: pd.DataFrame, preds: Iterable, id_col="id", target_col="label",
                                   path: str = "submission_classification.csv") -> pd.DataFrame:
    sub = pd.DataFrame({id_col: test_df[id_col].values, target_col: list(preds)})
    sub.to_csv(path, index=False)
    logger.success(f"Сабмит (классификация) сохранён: {path} shape={sub.shape}")
    return sub

def save_regression_submission(test_df: pd.DataFrame, preds: Iterable, id_col="id", target_col="target",
                               path: str = "submission_regression.csv") -> pd.DataFrame:
    sub = pd.DataFrame({id_col: test_df[id_col].values, target_col: list(preds)})
    sub.to_csv(path, index=False)
    logger.success(f"Сабмит (регрессия) сохранён: {path} shape={sub.shape}")
    return sub

def save_text_submission(test_df: pd.DataFrame, texts: List[str], id_col="id", target_col="prediction",
                         path: str = "submission_text.csv") -> pd.DataFrame:
    sub = pd.DataFrame({id_col: test_df[id_col].values, target_col: texts})
    sub.to_csv(path, index=False)
    logger.success(f"Сабмит (текст) сохранён: {path} shape={sub.shape}")
    return sub
```

---

## 9) Утилиты (профилирование, сохранение, контроль версий) <a id="sec9"></a>

```markdown
<div style="border:1px solid #333;padding:14px;border-radius:8px;background:#0f0f0f;color:#ddd">
<h2>🛠️ 9. Утилиты</h2>
<p>Профилирование времени, сохранение артефактов, быстрый репорт об окружении.</p>
</div>
```

```python
from contextlib import contextmanager

@contextmanager
def timer(name: str):
    t0 = time.time()
    yield
    dt = time.time() - t0
    logger.info(f"[TIMER] {name}: {dt:.3f}s")

def save_pickle(obj: Any, path: str):
    import pickle
    ensure_dir(os.path.dirname(path) or ".")
    with open(path, "wb") as f: pickle.dump(obj, f)
    logger.success(f"Pickle сохранён: {path}")

def load_pickle_fast(path: str) -> Any:
    import pickle
    with open(path, "rb") as f: return pickle.load(f)

def environment_report():
    logger.info(f"Python: {sys.version.split()[0]}")
    logger.info(f"Platform: {sys.platform}")
    try:
        import torch; logger.info(f"Torch: {torch.__version__}, cuda={torch.cuda.is_available()}")
    except Exception: logger.warn("Torch: недоступен")
    for lib in ["lightgbm","xgboost","catboost","sklearn","pandas","numpy"]:
        try:
            m = __import__(lib); logger.info(f"{lib}: {m.__version__}")
        except Exception:
            logger.warn(f"{lib}: недоступен")

environment_report()
```

---

# ✅ Конец файла

Этот файл можно вставить **как есть** в `README.md` или Markdown-ячейку Jupyter.
Хочешь — сотри все `<div>…</div>` описания, и останется только **чистый рабочий код**.

Если нужно — сделаю рядом **чистую `.py` версию** того же (без HTML/Markdown вставок).
